# PLANMAP-Invenio publishing: parsing PLANMAP

In [1]:
PUB_DIR = 'data/publish_data/archive/'
ZIP_DIR = 'data/publish_data/zip/'

In [2]:
import os

# Local api package
import api

In [3]:
%env INVENIO_API_TOKEN YXNIdLNRhWUubopweSvyD6o5qp5zUSTRq9zUKGsmkQK7NsBg0uBZ53W0A07Z
# %env INVENIO_API_TOKEN MJg5k2AYs5hr1YSXVKH4IifydN70WjUwcjbgxiKlxUaX2ETgB0nma4FiKZfx

env: INVENIO_API_TOKEN=YXNIdLNRhWUubopweSvyD6o5qp5zUSTRq9zUKGsmkQK7NsBg0uBZ53W0A07Z


In [4]:
# Invenio server
#
hostname = 'data.europlanet-gmap.eu'
# hostname = '10.72.2.28:5000'
token = os.environ['INVENIO_API_TOKEN']

client = api.invenio.InvenioClient(hostname, token)

In [5]:
import api.planmap
import api.invenio

from importlib import reload
reload(api.invenio)
reload(api.planmap)

import os
import sys
import warnings
import traceback
from glob import glob

warnings.filterwarnings("ignore")


def publish_package(pkg_dir):
    pkg_name = os.path.basename(pkg_dir)
    _,body,mtype,label = pkg_name.split('-')
    body = ({'mer':'mercury', 'mar':'mars', 'moo':'moon'}).get(body.lower())
    pkg_zip = f"{ZIP_DIR}{body}/{pkg_name}.zip"

    assert os.path.exists(pkg_dir) and os.path.exists(pkg_zip)
    print("- Ingesting package:", pkg_name)

    try:
        payload = api.planmap.parse_package(pkg_dir)
        # payload.add_files(pkg_zip)

    except Exception as err:
        print("-- Wrong doing while parsing:", repr(err))
        traceback.print_exc(file=sys.stdout)
        return None

    else:
        if payload is None:
            print("-- Parsing failed because of missing basic information"
                  "Skipping this dataset")
        else:
            print("-- Parsing OK")

    print("-- Publishing it")
    try:
        # Create a draft
        #
        resp = client.create_draft(payload)

        # Publish it
        #
        client.publish_draft(resp['id'])

    except Exception as err:
        print("-- Wrong doing while publishing:", repr(err))
        traceback.print_exc(file=sys.stdout)

    else:
        print("- Publishing OK:", resp['id'])
        return resp
    
            
published_list = []

for body in ['mars','mercury','moon']:
    print("Processing packages in", body)

    for pkg_dir in glob(f"{PUB_DIR}/{body}/PM-*"):
        
        resp = publish_package(pkg_dir)
        if resp:
            published_list.append(resp['id'])

# resp = publish_package('/Users/chbrandt/Coisas/data/planmap/publish_data/archive/mercury/PM-MER-MS-H05_5cc')

# resp

Processing packages in mars
- Ingesting package: PM-MAR-MS-Crommelin
File PM-MAR-MS-Crommelin.pdf too big (> 10000000 bytes).I'm not including it.
File PM-MAR-MS-Crommelin_basemap.pdf too big (> 10000000 bytes).I'm not including it.
-- Parsing OK
-- Publishing it
- Publishing OK: nf64x-gz409
- Ingesting package: PM-MAR-C-Crommelin
-- Parsing OK
-- Publishing it
- Publishing OK: 15zkw-6qe98
- Ingesting package: PM-MAR-D-Oxia_Planum
-- Parsing OK
-- Publishing it
- Publishing OK: 4fkq9-gz056
- Ingesting package: PM-MAR-D-Crommelin
-- Parsing OK
-- Publishing it
- Publishing OK: recy9-eym37
- Ingesting package: PM-MAR-MS-Arsinoes
-- Parsing OK
-- Publishing it
- Publishing OK: 61yva-yej90
- Ingesting package: PM-MAR-SI-Crommelin
-- Wrong doing while parsing: ValueError('could not convert string to float: \'4d56\\\'46.68"N\'')
Traceback (most recent call last):
  File "/var/folders/b1/frq3gywj3ljfqrf1yc7zk06r0000gn/T/ipykernel_3020/3365593906.py", line 27, in publish_package
    payload = 

In [6]:
# Open record's page
#
import webbrowser

webbrowser.open(resp['links']['self_html'])

TypeError: 'NoneType' object is not subscriptable